<a href="https://colab.research.google.com/github/GeoffreyKimani/InvetorySystem-VueJS-/blob/master/house_expenditure_anaylsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import pandas as pd

from google.colab import files

# upload file
data_file = files.upload()

Saving HouseHold Expenditure - Expenditure.csv to HouseHold Expenditure - Expenditure (2).csv


In [ ]:
# load data in dataframe
df = pd.read_csv(io.BytesIO(data_file["HouseHold Expenditure - Expenditure.csv"]), 
                 usecols=['Purchaser', 'Date', 'Cost', 'Category', 'Item'])

# remove rows with no entries based on date
df = df[df['Date'].notna()]

# displa
df

,Purchaser,Date,Cost,Category,Item
0,GK,30/08/2021,1600.0,Food,Sardines
1,NK,01/09/2021,4300.0,Food,Sardines 4 * 700 Milk 3 * 500
2,GK,02/09/2021,11400.0,House Items,2 Fantas + Nescafe coffee + 2 Instant porriage
3,GK,02/09/2021,1400.0,Food,Bread
4,NK,02/09/2021,3000.0,Food,1 KG Beef
5,GK,04/09/2021,12400.0,House Items,"Mayonnesse-3.5K, Peanut-5.5K, Prestige-2K, bre..."
6,GK,04/09/2021,3200.0,Food,Eggs - 1 Tray
7,Mx,04/09/2021,1800.0,Food,"Onions, Tomatoes"
8,MX,06/09/2021,16000.0,Food,"Meat(1.5kg)-4.5k, Maize Flour(5kg)-4K, Rice(5k..."
9,MX,07/09/2021,6700.0,Food,"Sardines(4)-2.8K, Potatoes(4kg)-1.2K, Milk(2)-..."


In [ ]:
# select all entries where names are unique and form their own dataframes
name_list = ['NK', 'MX', 'JO', 'GK']
purchaser_list = []

for name in name_list:
    new_df = name + '_df'
    new_df = df[df['Purchaser'] == name]
    purchaser_list.append(new_df)

purchaser_list

[  Purchaser        Date    Cost Category                           Item
 1        NK  01/09/2021  4300.0     Food  Sardines 4 * 700 Milk 3 * 500
 4        NK  02/09/2021  3000.0     Food                      1 KG Beef,
    Purchaser  ...                                               Item
 8         MX  ...  Meat(1.5kg)-4.5k, Maize Flour(5kg)-4K, Rice(5k...
 9         MX  ...  Sardines(4)-2.8K, Potatoes(4kg)-1.2K, Milk(2)-...
 10        MX  ...                                           Eggs(16)
 12        MX  ...  Beef(1kg)-3K, Cooking oil(1L)-2.8K, Bread-1.4K...
 16        MX  ...  Cornflakes (1kg) - 6k, Rwanda tea bag-3.4k, mi...
 17        MX  ...                                   Soap-Imperial -6
 18        MX  ...  Rice-(5kg)-5k, Meat(1kg)-3200, Maid bought oth...
 20        MX  ...  Beans-(4kg), Macaroni(3), Salt(0.5kg), Sugar(1...
 22        MX  ...  Cornflakes - 7500, milk - 1400, Kericho gold -...
 24        MX  ...  Rice(2kg) - 1600, Beef(1kg) - 3200, Tomatoes -...
 25       

In [ ]:
# for each of the dfs find the total spent and make a graph
for purchase in purchaser_list:
    # cost
    print(purchase['Purchaser'] , " spent ", purchase['Cost'].sum())

1    NK
4    NK
Name: Purchaser, dtype: object  spent  7300.0
8     MX
9     MX
10    MX
12    MX
16    MX
17    MX
18    MX
20    MX
22    MX
24    MX
25    MX
26    MX
29    MX
31    MX
32    MX
36    MX
38    MX
41    MX
Name: Purchaser, dtype: object  spent  157650.0
33    JO
Name: Purchaser, dtype: object  spent  4900.0
0     GK
2     GK
3     GK
5     GK
6     GK
11    GK
13    GK
14    GK
15    GK
19    GK
21    GK
23    GK
27    GK
28    GK
30    GK
34    GK
35    GK
37    GK
39    GK
40    GK
42    GK
Name: Purchaser, dtype: object  spent  110600.0
